In [1]:
import geopandas as gpd
import pandas as pd
import json
import copy
import os

### Konfiguration

In [77]:
state = 'MI'

In [78]:
pos = {
    'IA': {
        'name': 'Iowa',
        'scale': 2530,
        'translate': [80, 310],
        'arrow': 2
    },
    'PA': {
        'name': 'Pennsylvania',
        'scale': 4000,
        'translate': [-800, 380],
        'arrow': 2
    },
    'WI': {
        'name': 'Wisconsin',
        'scale': 2500,
        'translate': [-20,410],
        'arrow': 2
    },
    'MI': {
        'name': 'Michigan',
        'scale': 2070,
        'translate': [-90,370],
        'arrow': 2
    }
}

### Daten runterfiltern

In [79]:
d1 = gpd.GeoDataFrame(pd.read_pickle('temp/windofchange.pkl'))
d1

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
0,13027,Brooks,GA,"MULTIPOLYGON (((-83.73616 31.03768, -83.57396 ...",Trump,3.233335,16245.0,both,NaN
1,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",Trump,1.497509,7054.0,both,NaN
2,51683,Manassas,VA,"MULTIPOLYGON (((-77.52666 38.73320, -77.49545 ...",Harris,4.670204,42696.0,both,NaN
3,56021,Laramie,WY,"MULTIPOLYGON (((-105.27824 41.65666, -104.6533...",Trump,2.966682,100984.0,both,NaN
4,13135,Gwinnett,GA,"MULTIPOLYGON (((-84.27117 33.95980, -84.26265 ...",Harris,0.949814,983526.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3225,9007,NaN,NaN,None,Harris,-34.826903,NaN,right_only,Nikolai
3226,9008,NaN,NaN,None,Harris,NaN,NaN,right_only,Nikolai
3227,9009,NaN,NaN,None,Harris,-32.919680,NaN,right_only,Nikolai
3228,9010,NaN,NaN,None,Harris,NaN,NaN,right_only,Nikolai


In [80]:
d2 = d1[d1.state == state]
d2

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
16,26105,Mason,MI,"MULTIPOLYGON (((-86.51470 44.05812, -86.42987 ...",Trump,-0.978332,29159.0,both,NaN
30,26007,Alpena,MI,"MULTIPOLYGON (((-83.88848 44.85663, -83.88122 ...",Trump,0.742976,28904.0,both,NaN
71,26001,Alcona,MI,"MULTIPOLYGON (((-83.88848 44.85646, -83.31627 ...",Trump,1.624016,10489.0,both,NaN
175,26039,Crawford,MI,"MULTIPOLYGON (((-84.84731 44.85804, -84.37174 ...",Trump,1.518220,13538.0,both,NaN
178,26143,Roscommon,MI,"MULTIPOLYGON (((-84.85093 44.51107, -84.37064 ...",Trump,1.478379,23863.0,both,NaN
...,...,...,...,...,...,...,...,...,...
2965,26035,Clare,MI,"MULTIPOLYGON (((-85.08740 44.16424, -84.85170 ...",Trump,1.360920,31316.0,both,NaN
3083,26033,Chippewa,MI,"MULTIPOLYGON (((-85.23787 46.75570, -85.17304 ...",Trump,0.707057,36264.0,both,NaN
3125,26159,Van Buren,MI,"MULTIPOLYGON (((-86.35622 42.25417, -86.28445 ...",Trump,1.596400,75798.0,both,NaN
3128,26049,Genesee,MI,"MULTIPOLYGON (((-83.93207 43.22038, -83.69562 ...",Harris,2.579320,401522.0,both,NaN


In [81]:
d2.to_file('temp/'+state+'.geojson', driver='GeoJSON')

In [82]:
!geo2topo temp/{state}.geojson>temp/{state}.topo.json

### Arrowmap

In [83]:
with open('temp/'+state+'.topo.json', 'r') as f:
    stateData = json.load(f)

In [84]:
with open('specs/arrow-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [85]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']
spec_mw['signals'][0]['value'] = pos[state]['arrow']

In [86]:
folder_path = 'temp/specs/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [87]:
with open('temp/specs/arrow-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

### Bubblemap

In [88]:
with open('specs/bubble-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [89]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']

In [90]:
with open('temp/specs/bubble-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

In [91]:
!vg2svg temp/specs/arrow-{state}-mw.vg.json svgs/arrow-{state}-mw.svg
!vg2svg temp/specs/bubble-{state}-mw.vg.json svgs/bubble-{state}-mw.svg

(node:77073) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
(node:77075) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
